# PileCore

______________________________________________________________________

**Authors: Thijs Lukkezen & Robin Wimmers**

This notebook shows how to perform a pile bearing calculation with PileCore for multiple CPTs.

The structure of the tutorial is as follows:

- [Project definition](#Input-definition)
- [Download CPT's in project](#Download-CPTs-in-project)
- [Classify CPTs](#Classify-CPTs)
- [Call PileCore-API](#Call-PileCore-API)
- [View Results](#View-Results)
- [Report](#Report)
- [Grouper](#Grouper)
- [View Grouper Results](#View-Grouper-Results)

As usual, we start to install and import the libraries that are needed for this notebook.

In [ ]:
import datetime
import os
import io
from typing import Any, Dict, Mapping, Tuple, List
import logging

import numpy as np
import pandas as pd
import pygef
from matplotlib import pyplot as plt
from nuclei.client import NucleiClient
from pygef.cpt import CPTData
from pypilecore.input import (
    create_multi_cpt_payload,
    create_multi_cpt_report_payload,
    create_grouper_payload,
    create_grouper_report_payload,
)
from pypilecore.common.norms import Norms
from pypilecore import create_basic_pile
from pypilecore import api
from pypilecore.results import (
    MultiCPTCompressionBearingResults,
    GrouperResults,
    CasesMultiCPTBearingResults,
    CasesGrouperResults,
)
from pypilecore.viewers import (
    ViewerCptResults,
    ViewerCptResultsPlanView,
    ViewerResultsPerCptTable,
    ViewerGrouperResultsPerCptTable,
    ViewerCptResultsOverview,
)
from tqdm import tqdm

pd.set_option("display.max_columns", None)
logging.getLogger().setLevel(logging.INFO)

### Start a Nuclei client session

In the next cell we will create a nuclei-client with a session that takes care of the
authentication and communication with the Nuclei server.

You will need to provide your user token, which can be obtained by login in to the [nuclei website](https://nuclei.cemsbv.io/) with your personal credentials and going to the "API Access Tokens" section.

<div style="background: #f2ed4c;
            width: 100%;
            color: black;
            text-align: center;">
<b>USER INPUT REQUIRED BELOW:<b>
</div>

In [ ]:
# os.environ["NUCLEI_TOKEN"] = "<YOUR TOKEN>"

client = NucleiClient()

# Input definition

#### General input

In [ ]:
# General input data
project_id = 21305
project_remark = "Voorbeeld Notebook"  # Optional additional information
author = "N. Uclei"
project_name = "Automated pile design"

#### Norms input

In [ ]:
# ** NEN9997-1 selection
# Select the norms to be used in the calculation of bearing capacity of piles
# Accepted values are "2017" and "2025"
nen_9997_1_version = "2017"

# ** CUR236 selection
# Select the norm to be used in the calculation of uplift capacity of piles
# Accepted values are "2023"
cur_236_version = "2023"

# Create a norms object to be passed to the calculation endpoint
norms = Norms(
    nen_9997_1=nen_9997_1_version,
    cur_236=cur_236_version
)

#### Soil Investigation input

In [ ]:
# Specify CPT selection

# ** cpt_selection
# Specify a list of CPTs names (i.e. BRO ID).
cpt_selection = [
    "CPT000000200908",
    "CPT000000200909",
    "CPT000000112268",
    "CPT000000112278",
    "CPT000000200904",
    "CPT000000200905",
    "CPT000000112265",
    "CPT000000200906",
    "CPT000000200907",
]


# ** classify_metode:
# Metode used to classify CPT data.
# Accepted values: ["beenJefferies", "machineLearning", "nen", "table", "robertson", "ntype"]
classify_method = "ntype"

#### Geometry input

In [ ]:
# ** pile_tip_levels_nap
# These are the levels (w.r.t. NAP) at which the bearing capacities will be calculated
# It can be a list, or any other sequence, such as a numpy array.
pile_tip_levels_nap = np.arange(-14, -18, -1.0)

# ** pile_head_level_nap
# The level of the pile-head [m] w.r.t. NAP.
# Must be a number, or the string "surface". In the latter case, the pile-head will be
# situated at the level of the original CPT surface level.
pile_head_level_nap = "surface"

# ** groundwater_level
# The groundwater-level in the project (w.r.t. NAP). Will have an impact on the
# classification and on the calculated soil-stresses.
# When None, the water-level of the CPTs is used.
groundwater_level_nap = None

#### Excavation input

In [ ]:
# ** excavation_depth_nap:
# The depth [m w.r.t. NAP] of the service-level after excavation.
# Has to be below the origin al service-level of the CPT.
excavation_depth_nap = None

# ** excavation_param_t:
# Required when providing an excavation_depth. The values can be:
# - 1.0: if installation is not low in vibration (niet-trillingsarm) and piles are installed after excavating
# - 0.5: (wortel-methode) if piles have been installed before excavation or installation is low-vibrating
excavation_param_t = 1.0

# ** excavation_stress_reduction_method:
# Method used to calculate the stress reduction due to the excavation applied to the effective and total stresses.
# Only used when `excavation_depth_nap` is different than `None`. It can be:
# - "constant": The stress reduction below the excavation is constant with depth. The stress reduction is equal to
#       the original effective stress (i.e. before the excavation) at the excavation depth.
# - "begemann": The stress reduction below the excavation decreases with depth according to the Begemann method.
excavation_stress_reduction_method = "constant"

# ** excavation_width:
# Width of the excavation [m].
# Used to calculate the stress reduction due to the excavation if the Begemann method is selected.
# Accepted values: > 0.0
excavation_width = None

# ** excavation_edge_distance:
# Distance from the edge of the excavation to the pile centerline [m].
# Used to calculate the stress reduction due to the excavation if the Begemann method is selected.
# Accepted values: >= 0.0 and <= 0.5 * excavation_width
# Note: 0.0 means pile at edge of excavation, 0.5 * excavation_width means pile at center of excavation.
excavation_edge_distance = None

#### Construction and loads input

In [ ]:
# ** stiff_construction:
# The stiffness of the construction has an influence on the value of xi.
# Accepted values: [True, False]
stiff_construction = False

# ** pile_load_sls:
# The load on the pile-head [kN] (in serviceability state / SLS)
# if None, A fraction of the bearing capacity is used as load to calculate the pile settlement + pile shortening
# Accepted values: > 0.0
pile_load_sls = None

# ** relative_pile_load:
# The fraction of the bearing capacity that is mentioned at `pile_load_sls`.
# A value of 0.7 translates to 70% of the characteristic bearing capacity.
# Accepted values: between 0.0 and 1.0
relative_pile_load = 0.7

# ** soil_load_sls:
# The overburden-pressure at surface-level [kPa]
# If None, the default is 0.0
soil_load_sls = 0

# ** pile_load_uls:
# The required load on the pile-head [kN] (ULS).
pile_load_uls = 100

#### Shaft Friction input

In [ ]:
# ** negative_shaft_friction
# A fixed value of the negative-shaft friction force [kN].
# All other negative-friction settings will be ignored if you fill this.
# Accepted values: > 0.0
negative_shaft_friction = None

# ** friction_range_strategy
# Determination of the positive and negative friction strategy
# For manual determination use 'manual', else 'lower_bound' or 'settlement_driven'
friction_range_strategy = "lower_bound"

# Only used with friction range strategy "manual"
# ** fixed_negative_friction_range_nap
# Define a fixed range for the negative friction w.r.t. NAP
# e.g.: (0.0, -5.0)
fixed_negative_friction_range_nap = None

#** fixed_positive_friction_range_nap
# Define a fixed range for the positive friction w.r.t. NAP
# e.g.: (-5.0, "ptl") or (-5.0, -15.0) etc.
fixed_positive_friction_range_nap = None

# ** Overrule ranges for specific CPTs
# If desired, you can also specify friction ranges per CPT
# The values provided below will overwrite the friction ranges for those specific CPTs.

## ** individual_negative_shaft_friction
# Dictionary with key cpt name and value the negative shaft friction of that cpt in kN
# e.g.: {"S03" : 150.0}
individual_negative_shaft_friction: Mapping[Any, float] = {}

# ** individual_positive_friction_range_nap
# dictionary with key cpt name and value a tuple with upper and lower bound
# e.g.: {"S03": (-17.5, 'ptl')}
individual_positive_friction_range_nap: Mapping[Any, Tuple[float, str]] = {}

# ** individual_negative_friction_range_nap
# dictionary with key cpt name and value a tuple with upper and lower bound
# e.g.: {"S03": (-5.5, -17.5)}
individual_negative_friction_range_nap: Mapping[Any, Tuple[float, str]] = {}



#### OCR input

In [ ]:
# ** ocr
# The Over-Consolidation-Ratio [-] of the foundation layer.
# If None, an OCR of 1.0 is assumed.
ocr = None

# ** Overrule OCR for specific CPTs
# If desired, you can also specify OCR values per CPT
# The values provided below will overwrite the OCR for those specific CPTs.

# ** individual_ocr
# dictionary with key cpt name and OCR value
# e.g.: {"S03": 1.5}
individual_ocr: Mapping[Any, float] = {}

#### Safety Factors input

In [ ]:
# ** gamma_r_b
# Safetyfactor on the pile-tip bearing capacity
# If None, the default is 1.2
gamma_r_b = 1.2

# ** gamma_r_s
# Safetyfactor on the sleeve-friction bearing capacity
# If None, the default is 1.2
gamma_r_s = 1.2

# ** gamma_f_nk
# Safetyfactor on the negative friction
# Note: Use 1.4 if bottom negative friction ≠ positive friction
# If None, the default is 1.0
gamma_f_nk = 1.0

# ** overrule_xi
# Overrule the xi-values used in the calculation of group effects.
# Can set a single value for xi3, xi4 and xi4_single by providing a number
# OR specify individual values by providing a dictionary with keys 'xi3', 'xi4' and 'xi4_single'.
# Example: {"xi3" : 1.39, "xi4" : 1.27, "xi4_single" : 1.2}
overrule_xi = None

## Pile input
---


In [ ]:
# ** pile_name
# The name of the pile (For potential plotting purposes)
pile_name = None

### Standard piles

Choose a `reference` to define a standard pile according to the NEN9997-1 or CUR236 definitions.

You can also leave the standard pile definition empty and define all pile attributes manually.

#### NEN9997-1
![NEN9997-1a](https://crux-nuclei.com/api/pilecore/v4/static?filename=standard_piles-NEN9997-1_2025.png)

#### CUR-236
![CUR236a](https://crux-nuclei.com/api/pilecore/v4/static?filename=standard_piles-CUR236_2023.png)

In [ ]:
# Standard Pile definition

# ** reference
# The choice of pile from one of the tables above.
# Accepted values: 
#   NEN9997-1:2025  -> ["B1", "B2", "B3", "B4", "B5", "B6", "B7", "B8", "S1", "S2", "S3", "S4", "S5", "S6", "S7", "H1", "H2"]
#   CUR-236:2023    -> ["AA1", "AA2", "AB1", "AB2", "AC", "AD", "AE"]
reference = "B3"

### Pile Geometry
![pile_geometry_definitions](https://crux-nuclei.com/api/pilecore/v4/static?filename=pile_geometry_definitions.png)

In [ ]:
# ** pile_shape
# The shape of the pile
# Accepted values: ["round", "rectangle"]
pile_shape = "round"

# ** height_base
# Height of pile base [m]. If None, a pile with constant dimension is inferred.
height_base = None

### Rectangular pile dimensions

Only fill these values if the `pile_shape` == "rectangle". Otherwise they are ignored

The pile geometry is defined as a "core" and "shell" segment. First we define the "core",
which is a required part that should always be there for a rectangular pile.

In [ ]:
# ** core_secondary_dimension
# Largest cross-sectional dimension of the core component [m].
core_secondary_dimension = 0.40

# ** core_tertiary_dimension
# Smallest cross-sectional dimension of the core component [m].
# If None, a square core is inferred.
core_tertiary_dimension = None

# Base dimensions (optional)
# --------------------------
# Optionally, we can define a "base" segment. This segment should have larger dimensions than the core.
# Make sure to also fill a value for the `height_base` (in the general geometry cell above).

# ** base_secondary_dimension
# Largest cross-sectional dimension of the widened-base component [m].
base_secondary_dimension = None

# ** base_tertiary_dimension
# Smallest cross-sectional dimension of the widened-base component [m].
# If None, a square base is inferred.
base_tertiary_dimension = None

### Round pile dimensions

Only fill these values if the `pile_shape` == "round". Otherwise they are ignored

The pile geometry is defined as a "core" and "shell" segment. First we define the "core",
which is a required part that should always be there for a round pile.

In [ ]:
# ** core_diameter
# Diameter of the core component [m].
core_diameter = 0.40

# Base dimension (optional)
# --------------------------
# Optionally, we can define a "base" segment. This segment should have larger dimensions than the core.
# Make sure to also fill a value for the `height_base` (in the general geometry cell above).

# ** base_diameter
# Diameter of the widened-base [m].
# If null, a pile with constant diameter (diameter_base) is inferred.
base_diameter = None

### Optionally overwriteable pile input

If you want to use the default pile specifications you can ignore this section. Any
value provided will overwrite the default value.

In [ ]:
# Optional Input

# ** pile_material
# The material name of the pile. If a standard pile was selected, the material is
# inferred, but can be overwritten. Default materials are: "concrete", "steel", "wood".
# Custom materials can be defined below.
pile_material = None

# ** custom_material
# A custom material definition. Assign the "name" property as `pile_material` to use it.
# example:
# custom_material = {
#     "name": "custom_material",
#     "elastic_modulus": 15e3,    # [MPa]
#     "color": "#ff0000",        # Hexadecimal color
# }
custom_material = None

# ** installation
# The installation method of the pile. The value is inferred from the pile reference, but can be overwritten.
# possible values: ["driven", "screwed", "excavated", "vibrated", "pressed", "jetted"], 
# of which "excavated", "pressed", "jetted" and "screwed" are considered low-vibrating
installation = None

# ** settlement_curve
# Settlement lines for figures 7.n and 7.o of NEN-9997-1 As defined in table 7.c of
# NEN-9997-1. The value is inferred from the pile_type_specifications, but can be
# overwritten
settlement_curve = None

# ** adhesion
# Optional adhesion value, use it if the pile shaft has undergone a special treatment.
# Examples:
# - adhesion = 50 kN/m2 for synthetic coating
# - adhesion = 20 kN/m2 for bentonite
# - adhesion = 10 kN/m2 for bitumen coating
# See 7.3.2.2(d) of NEN 9997-1 for examples.
adhesion = None  # kPa

# ** alpha_p
# Alpha p factor used in pile tip resistance calculation. The value is inferred from the
# pile_type_specifications, but can be overwritten.
alpha_p = None

# ** alpha_s_clay
# Alpha s factor for soft layers used in the positive friction calculation. If None the
# factor is determined as specified in table 7.d of NEN 9997-1.
# examples:
# alpha_s_clay = {"use_constant_value" : True, "constant_value" : 0.5}
# alpha_s_clay = {"use_constant_value" : False}
alpha_s_clay = None

# ** alpha_s_sand
# Alpha s factor for coarse layers used in the positive friction calculation. The value
# is inferred from the pile_type_specifications, but can be overwritten.
alpha_s_sand = None

# ** beta_p
# Beta_p used in pile tip resistance calculation as per NEN 9997-1 7.6.2.3 (h). The
# value is inferred from the pile dimension properties, but can be overwritten
beta_p = None

# ** pile_tip_factor_s
# Factor s used in pile tip resistance calculation as per NEN 9997-1 7.6.2.3 (h). The
# value is inferred from the pile dimensions and soil properties, but can be overwritten.
pile_tip_factor_s = None

# ** is prefab
# Determines whether the pile is prefabricated or cast in place. 
# The value is inferred from the reference, but can be overwritten.
# Accepted values: [True, False, None]
is_prefab = None

# ** is_auger
# Determines weather the pile is an auger pile or not. The value is inferred
# from the pile_type_specifications, but can be overwritten.
# Accepted values: [True, False, None]
is_auger = None

# ** is_low_vibrating
# Determines weather the pile has an installation type with low vibration. The value is
# inferred from the pile_type_specifications, but can be overwritten.
# Accepted values: [True, False, None]
is_low_vibrating = None

# ** is_open_ended
# Determines whether the pile is open-ended or closed-ended. The value if inferred 
# from the reference for standard piles but can be overwritten.
# Accepted values: [True, False, None]
is_open_ended = None

# ** negative_fr_delta_factor
# factor * φ = δ. This parameter will be multiplied with phi to get the delta parameter
# used in negative friction calculation according to NEN-9997-1 7.3.2.2 (e). Typically
# values are 1.0 for piles cast in place, and 0.75 for other pile types. The value is
# inferred from the pile_type_specifications, but can be overwritten.
negative_fr_delta_factor = None

# ** qc_z_a_lesser_1m
# Maximum cone resistance `qc` value allowed for layers with thickness < 1m in
# the calculation of positive skin friction resistance. This value is used to compute the
# trimmed (chamfered) `qc;z;a` values according to NEN 9997-1+C2:2017 7.6.2.3.(10)(i).
# It must be less or equal than `qc_z_a_greater_1m`. If None, then 12 MPa is used.
qc_z_a_lesser_1m = None

# ** qc_z_a_greater_1m
# Maximum cone resistance `qc` value allowed for layers with thickness >= 1m in
# the calculation of positive skin friction resistance. This value is used to compute the
# trimmed (chamfered) `qc;z;a` values according to NEN 9997-1+C2:2017 7.6.2.3.(10)(i)
# It must be greater or equal than `qc_z_a_lesser_1m`. If None, then 15 MPa is used.
qc_z_a_greater_1m = None

# ** qb_max_limit
# Maximum value allowed for the pile tip resistance `qb_max` obtained according to
# NEN 9997-1+C2:2017 7.6.2.3.(10)(e). Note that is value will be used as a limit for `qb_max`
# unless `use_almere_rules` is `true` and a higher value than 12 MPa is specified.
# If None, then 15 MPa is used.
qb_max_limit = None

### Create Pile

In [ ]:
pile = create_basic_pile(
    pile_name=pile_name,
    reference=reference,
    installation=installation,
    pile_shape=pile_shape,
    height_base=height_base,
    core_secondary_dimension=core_secondary_dimension,
    core_tertiary_dimension=core_tertiary_dimension,
    base_secondary_dimension=base_secondary_dimension,
    base_tertiary_dimension=base_tertiary_dimension,
    core_diameter=core_diameter,
    base_diameter=base_diameter,
    pile_material=pile_material,
    custom_material=custom_material,
    settlement_curve=settlement_curve,
    adhesion=adhesion,
    alpha_p=alpha_p,
    alpha_s_clay=alpha_s_clay,
    alpha_s_sand=alpha_s_sand,
    beta_p=beta_p,
    pile_tip_factor_s=pile_tip_factor_s,
    is_auger=is_auger,
    is_low_vibrating=is_low_vibrating,
    negative_fr_delta_factor=negative_fr_delta_factor,
    qc_z_a_lesser_1m=qc_z_a_lesser_1m,
    qc_z_a_greater_1m=qc_z_a_greater_1m,
    qb_max_limit=qb_max_limit,
)

In [ ]:
pile.geometry.plot()

### Other input

Some other input values.

Look [here](https://www.almere.nl/fileadmin/user_upload/Richtlijnen_Constructie_Gem._Almere_vanaf_01-01-2017_versie_3.0a.pdf) for more info about the Almere rules.

In [ ]:
# ** use_almere_rules
# If set to True the contribution, produced by the positive shaft friction, to the total
# bearing capacity is limited to at most 75% the contribution provided by the pile tip.
use_almere_rules = None

# ** apply_qc3_reduction
# Determines if the reduction on the qc;III trajectory for auger piles should be applied
# conform 7.6.2.3(e) of NEN 9997-1. If None, the value is inferred from the pile type:
# only True for auger piles. If a boolean is provided, the qc3 reduction is/isn't
# applied, regardless of the pile type.
apply_qc3_reduction = None

### Report content

These values define the content of the report

In [ ]:
# ** group_results_content
# Whether or not to add a section with the results of all CPTs considered as one
# statistical group.
# Accepted values: [True, False]
group_results_content = True

# ** individual_cpt_results_content
# Whether or not to add a separate result section for each individual CPT.
# Accepted values: [True, False]
individual_cpt_results_content = True

# ** result_summary_content
# Whether or not to add a summary of all results in the beginning of the report.
# Accepted values: [True, False]
result_summary_content = True

<div style="background: #f2ed4c;
            width: 100%;
            color: black;
            text-align: center;">
<b>END USER INPUT<b>
</div>

After this point, modifications are for expert users

#### Download CPTs in project

In [ ]:
# Get CPTs
# loop over the cpt id's and fetch file from BRO
cptdata_objects: List[CPTData] = []
for file_metadata in tqdm(cpt_selection, desc="Download CPT's from BRO"):
    # download CPT from BRO
    response = client.session.get(
        url=f"https://publiek.broservices.nl/sr/cpt/v1/objects/{file_metadata}"
    )
    if not response.ok:
        print(
            f"RuntimeError: {file_metadata} could not be donwloaded from de BRO server. \n Statuse code: {response.status_code}"
        )
        continue

    cpt = pygef.read_cpt(io.BytesIO(response.content))
    object.__setattr__(cpt, "alias", file_metadata)
    cptdata_objects.append(cpt)

#### Classify CPTs

In [ ]:
classify_tables: Dict[str, dict] = {}

for i, cpt in tqdm(enumerate(cptdata_objects), desc="Classify CPT's"):
    # remove nan data
    data = cpt.data.drop_nulls()

    # classify CPT with CPTCore
    payload = {
        "aggregateLayersPenalty": 5,
        "minimumSegmentLength": 5,
        "data": {
            "coneResistance": data.get_column("coneResistance").clip(0, 50).to_list(),
            "correctedPenetrationLength": data.get_column("depth").to_list(),
            "localFriction": data.get_column("localFriction").clip(0, 50).to_list(),
        },
        "verticalPositionOffset": cpt.delivered_vertical_position_offset,
        "x": cpt.delivered_location.x,
        "y": cpt.delivered_location.y,
    }
    if "porePressureU2" in data.columns:
        payload["data"]["porePressureU2"] = (
            data.get_column("porePressureU2").clip(0, 50).to_list(),
        )[0]

    response = client.session.post(
        f"https://crux-nuclei.com/api/cptcore/v1/classify/{classify_method}",
        json=payload,
    )
    if not response.ok:
        cptdata_objects.pop(i)
        print(
            f"RuntimeError: {file_metadata} could not be classified. \n Statuse code: {response.status_code}"
        )
        continue
    classify_tables[cpt.alias] = response.json()

#### Call PileCore-API

In [ ]:
# Get results

multi_cpt_payload, results_passover = create_multi_cpt_payload(
    cptdata_objects=cptdata_objects,
    classify_tables=classify_tables,
    groundwater_level_nap=groundwater_level_nap,
    friction_range_strategy=friction_range_strategy,
    excavation_depth_nap=excavation_depth_nap,
    individual_negative_shaft_friction=individual_negative_shaft_friction,
    individual_negative_friction_range_nap=individual_negative_friction_range_nap,
    individual_positive_friction_range_nap=individual_positive_friction_range_nap,
    pile=pile,
    excavation_param_t=excavation_param_t,
    excavation_stress_reduction_method=excavation_stress_reduction_method,
    excavation_width=excavation_width,
    excavation_edge_distance=excavation_edge_distance,
    pile_head_level_nap=pile_head_level_nap,
    pile_load_sls=pile_load_sls,
    pile_tip_levels_nap=list(pile_tip_levels_nap),
    relative_pile_load=relative_pile_load,
    apply_qc3_reduction=apply_qc3_reduction,
    negative_shaft_friction=negative_shaft_friction,
    fixed_negative_friction_range_nap=fixed_negative_friction_range_nap,
    fixed_positive_friction_range_nap=fixed_positive_friction_range_nap,
    norms=norms,
    gamma_f_nk=gamma_f_nk,
    gamma_r_b=gamma_r_b,
    gamma_r_s=gamma_r_s,
    overrule_xi=overrule_xi,
    soil_load_sls=soil_load_sls,
    use_almere_rules=False,
    stiff_construction=stiff_construction,
    cpts_group=None,
    ocr=ocr,
    individual_ocr=individual_ocr
)

api_response = api.get_multi_cpt_api_result(client=client, payload=multi_cpt_payload)

multi_bearing_results = MultiCPTCompressionBearingResults.from_api_response(
    response_dict=api_response,
    cpt_input=results_passover,
)

## Results

In [ ]:
# Optionally define a case name for the results (to be shown in the interactive viewers)
case_name = "Default Case"

In [ ]:
# Parse the results into a CasesMultiCPTBearingResults
results_per_case = {
    case_name: multi_bearing_results,
}

cpt_locations = {cpt.alias: cpt.delivered_location for cpt in cptdata_objects}

multi_cpt_results_cases = CasesMultiCPTBearingResults(
    results_per_case=results_per_case, cpt_locations=cpt_locations
)

### CPT Results Table

In [ ]:
case_multi_results_table_viewer = ViewerResultsPerCptTable(
    cases_cpt_results=multi_cpt_results_cases
)
case_multi_results_table_viewer.display()

# You can also get the currently displayed table as a pandas DataFrame:
df_multi_results_selection = case_multi_results_table_viewer.to_pandas()

### CPT Results Viewer

In [ ]:
viewer_cpt_results = ViewerCptResults(results_cases=multi_cpt_results_cases)
viewer_cpt_results.display()

### CPT Results Plan View

In [ ]:
viewer_cpt_results_plan_view = ViewerCptResultsPlanView(
    results_cases=multi_cpt_results_cases
)
viewer_cpt_results_plan_view.display()

### Single CPT inspection

In [ ]:
viewer_cpt_results_overview = ViewerCptResultsOverview(
    results_cases=multi_cpt_results_cases
)
viewer_cpt_results_overview.display()

In [ ]:
# Plot the variation of an attribute between group and single result for a single pile tip level

multi_bearing_results.boxplot(attribute="k_v_1", show_sqrt=True);

## Report

In [ ]:
# Create report

# Close all open plots to save memory
plt.close("All")

multi_cpt_report_payload = create_multi_cpt_report_payload(
    multi_cpt_payload=multi_cpt_payload,
    project_name=project_name,
    project_id=str(project_id),
    author=author,
    date=datetime.date.today().strftime("%d-%m-%y"),
    group_results_content=group_results_content,
    individual_cpt_results_content=individual_cpt_results_content,
    result_summary_content=result_summary_content,
)

report = api.get_multi_cpt_api_report(client=client, payload=multi_cpt_report_payload)

with open(f"{project_name}_report.pdf", "wb") as f:
    f.write(report)

## Grouper

In [ ]:
# Create groups
grouper_payload = create_grouper_payload(
    cpt_results_dict=multi_bearing_results.cpt_results.cpt_results_dict,
    building_polygon=None,
    cpt_grid_rotation=0.0,
    gamma_bottom=gamma_r_b,
    gamma_shaft=gamma_r_s,
    include_centre_to_centre_check=False,
    stiff_construction=stiff_construction,
    resolution=1,
)

group_response = api.get_groups_api_result(client=client, payload=grouper_payload)

group_result = GrouperResults.from_api_response(
    group_response,
    pile_load_uls=pile_load_uls,
    multi_cpt_bearing_results=multi_bearing_results,
)

## View Grouper Results

In [ ]:
group_results_cases = CasesGrouperResults(
    results_per_case={case_name: group_result}, cpt_locations=cpt_locations
)

In [ ]:
case_grouper_results_table_viewer = ViewerGrouperResultsPerCptTable(
    group_results_cases=group_results_cases
)
case_grouper_results_table_viewer.display()

# You can also get the currently displayed table as a pandas DataFrame:
df_grouper_results_selection = case_grouper_results_table_viewer.to_pandas()

In [ ]:
viewer_cpt_group_results = ViewerCptResults(results_cases=group_results_cases)
viewer_cpt_group_results.display()

In [ ]:
group_results_plan_viewer = ViewerCptResultsPlanView(results_cases=group_results_cases)
group_results_plan_viewer.display()

In [ ]:
group_results_cases.results_per_case[case_name].cpt_results.plot(
    projection=None, hue="category"
);

In [ ]:
group_results_cases.results_per_case[case_name].cpt_results.plot(
    hue="category", pile_load_uls=pile_load_uls
);

In [ ]:
viewer_cpt_grouper_results_overview = ViewerCptResultsOverview(
    results_cases=group_results_cases
)
viewer_cpt_grouper_results_overview.display()

## Grouper Report

In [ ]:
# Create report

# Close all open plots to save memory
plt.close("All")

multi_cpt_report_payload = create_grouper_report_payload(
    grouper_payload=grouper_payload,
    grouper_response=group_response,
    project_name=project_name,
    project_id=str(project_id),
    author=author,
)

report = api.get_groups_api_report(client=client, payload=multi_cpt_report_payload)

with open(f"{project_name}_grouper_report.pdf", "wb") as f:
    f.write(report)